In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%config InlineBackend.figure_format ='retina'
    
    
import numpy as np


import eqstochsim as sm
import eqstochsim.models
import eqstochsim.eqphysics

from plotconf import matsettings

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import matplotlib.pyplot as plt

/opt/anaconda3/envs/YPML/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


# Intro to stochastic ground motion simulation - Pt. 1
### This notebook is intended to show the basic building blocks of a realistic earthquake ground motion model, principally following the general outline of Boore (2003).
*For clarity we will follow their equation notations as closely as we can, but inevitably there will be some mismatches.*
#### References
1. [Aki, K. (1967). Scaling law of earthquake spectrum. *J. Geophys. Res.* **72**, 1217-1231.](https://doi.org/10.1029/JZ072i004p01217) 
2. [Boore, D.M. (2003). Simulation of ground motion using the stochastic method. *Pure Appl. Geophys.* **160**, 635-676.](http://www.daveboore.com/pubs_online/boore_stochastic_method_pageoph.pdf)
3. [Brune, J. (1970). Tectonic stress and the spectra of seismic shear waves from earthquakes, *J. Geophys. Res.* **75**, 4997-5009.](https://doi.org/10.1029/JB075i026p04997)
4. [Brune, J. (1971). Correction, *J. Geophys. Res.* **76**, 5002.](https://doi.org/10.1029/JB076i020p05002) 
5. [Eshelby, J. D. (1957). The determination of the elastic field of an ellipsoidal inclusion, and related problems. *Proc. Royal Soc. A.* **241**, 376–396.](https://doi.org/10.1098/rspa.1957.0133)
6. [Kaneko, Y. and P.M. Shearer (2014). Seismic source spectra and estimated stress drop derived from cohesive-zone models of circular subshear rupture. *Geophys. J. Int.* **197**, 1002-1015.](https://doi.org/10.1093/gji/ggu030) 
7. [Shearer, P. M., R. E. Abercrombie, D. T. Trugman, and W. Wang (2019). Comparing EGF methods for estimating corner frequency and stress drop from P wave spectra, *J. Geophys. Res.* **124**, 3966–3986.]( https://doi.org/10.1029/2018JB016957)


### Overview of the Stochastic Method

The stochastic method convolves a series of parametric or functional models that describe the ground motion spectrum for arbitrary earthquake scenarios with random phases (Boore, 2003). As it will be mentioned many times, we define an "earthquake scenario" as the combination of an arbitrary earthquake source (seismic moment, stress drop), convolved with the response of a path, site and (if desired) instrument response for a given source-site propagation, and the physical properties of the medium (e.g., seismic velocity, density). <br><br> It is widely used to simulate higher-frequency ground motions, which are either simply not available from real data nor can be simulated using finite-difference methods (or other facier varients) due to computational cost. You can broadly separate recorded ground motion into three stages of the ray-path's journey from generation to the location/instrument of interest; source, path and site (Fig.1). 

<img src="./Figures/SPSFig.png" width="900" height="500" />

*Fig. 1 - adopted from [David Boore's presentation on source, path and site from observations.](https://www.daveboore.com/short_course_lectures_and_labs.html)*



### Generalized model of ground/instrument motion induced by an incident S-wave from an earthquake

The equation for the resultant far-field displacement spectrum for an arbitrary earthquake scenario is described by the following representitive equation (Boore, 2003):

#### $$Y(M_{0}, R, f) = E(M_{0}, f)P(R,f)G(f)I(f),\tag{1}$$

$E(M_{0}, f)$ is the source model, $M_{0}$ is seismic moment ($\mathrm{N \cdot m}$ or $\mathrm{dyne \cdot cm}$, $R$ is distance ($\mathrm{m}$ or $\mathrm{km}$) and $f$ is frequency ($\mathrm{Hz}$). $P(R,f)$ is the path model, $G(f)$ is the site model and $I(f)$ is the instrument response. Here we will only consider a point source model, assuming that the observational distances are $>>$ than the wavelengths emitted by the source (far-field spectrum is the dominant signal).

### Source model: $E(M_{0}, f)$

For simplicity we will use a simple relation between seismic moment and corner frequency that was developed by Brune(1970, 1970). This model assumes a circular crack model (Eshelby, 1957). We will also assume self-similarity, and an 'omega-squared' model, meaning the seismic moment and corner frequency are related by a constant $M_{0}f^{3}_{c} = c$ and the high frequency spectral fall-off is $f^{-2}$(Aki, 1967).

The shape of such a source model is represented by the following equation (Shearer *et al.*, 2019):

#### $$ E(M_{0}, f) = \frac{C\Omega_{0}}{(1+(f/f_c)^{\gamma n})^{\frac{1}{\gamma}}} = \frac{M_{0}}{(1+(f/f_c)^{\gamma n})^{\frac{1}{\gamma}}}\tag{2}$$

The constant (C) that scales the long period spectral level ($\Omega_0$ - displacment [$\mathrm{m s}$]) to $\mathrm{M_{0}}$ is implicitly linked to the particular model and sets of assumptions used to tie stress drop ($\Delta\sigma$), corner frequency ($f_c$) and seismic moment ($M_{0}$) to the far-field model. There are a number of these relations that are routinely used, but as mentioned earlier we use Brune (1970). We refer the reader to Shearer *et al.* (2019) for a deeper discussion. In this case we use $n = 2$, and $\gamma = 1$ for the shape of the spectrum. In addition, assuming a Brune (1970) relation, $C$ is given by:

#### $$C = \frac{4\pi V^{3}_{s}\rho r_0}{F \Theta_{\lambda\Phi}},\tag{3}$$

where $V_S$ and $\rho$ are the shear wave velocity ($\mathrm{m/s}$) and density ($\mathrm{kg m^{-3}}$) at the source, $F$ is an amplification factor of 2 (to account for free surface interactions), and $\Theta_{\lambda\Phi}$ is an empirically derived parameter (0.55, Boore and Boatwright, 1984) that accounts for the S-wave radiation pattern, averaged over the whole focal sphere.<br>

Furthermore, the relation between $\Delta\sigma$ ($\mathrm{Pa}$), $M_{0}$ ($\mathrm{N \cdot m}$) and $f_c$ ($\mathrm{Hz}$) is given by (Shearer et al., 2019):



#### $$ \Delta\sigma = \frac{7}{16}\left(\frac{f_{c}}{k V_{s}}\right)^{3}M_0,\tag{4}$$

for which we assume $k\approx0.37$ for the Brune (1970, 1971) model (Kaneko and Shearer, 2014). Finally, seismic moment ($\mathrm{M_{0}}$) is also related to the more commonly referenced moment magnitude ($\mathrm{M_{w}}$) scale (Hanks and Kanamori, 1979) by:

#### $$ \mathrm{M_{w}} = \frac{2}{3}\mathrm{log_{10}}(M_{0}) - 6.0\dot{3} \tag{5}$$

In [2]:
## Reference models for interactive plot
SD = np.array([0.1, 1, 10])
MW = np.arange(3, 10)
MO = sm.eqphysics.mo_from_mw(MW)
FC = sm.eqphysics.fc(3600, SD[1]*(1E6), MO)
F = np.round(np.geomspace(FC*0.01, FC*50), 4)
E = 10**sm.models.source_scf(F, np.log10(MO), FC, 1, 2)
FC_0 = sm.eqphysics.fc(3600, SD[0]*(1E6), MO)
FC_1 = sm.eqphysics.fc(3600, SD[2]*(1E6), MO)

In [3]:
@interact
def plot_source(mw=(3, 9, 0.5), sd=[1, 0.1, 10], vs=[3600, 3200, 2800, 2400], gam=[1,2], n=[2,1]):
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    ax[0].loglog(F, E, '--', color='grey', lw=2)
    ax[0].set_xlim([F.min().min()*0.9, F.max().max()*1.1])
    ax[0].set_ylim([E.min().min()*0.7, E.max().max()*2])
    ax[0].set_xlabel("Freq. [Hz]")
    ax[0].set_ylabel("Moment Rate [Nm]")
    
    ax[1].plot(FC_0, MO, '--', color='grey', label=r'$\Delta\sigma${} MPa'.format(SD[0]))
    ax[1].loglog(FC, MO, '--', color='grey', label=r'$\Delta\sigma${} MPa'.format(SD[1]))
    ax[1].plot(FC_1, MO, '--', color='grey', label=r'$\Delta\sigma${} MPa'.format(SD[2]))
    ax[1].set_xlabel(r"$f_{c}$ [Hz]")
    ax[1].set_ylabel(r"$M_0$")
    ax[1].legend()
    
    mo = sm.eqphysics.mo_from_mw(mw)
    fc = sm.eqphysics.fc(vs, sd*(1E6), mo)
    f = np.round(np.geomspace(fc*0.01, fc*50), 4)    
    ax[0].plot(f, 10**sm.models.source_scf(f, np.log10(mo), fc, gam, n), 'r', lw=2)
    ax[1].plot(fc, mo, 'ro')
    fig.tight_layout()
    
    

    

interactive(children=(FloatSlider(value=6.0, description='mw', max=9.0, min=3.0, step=0.5), Dropdown(descripti…

### Path model: $P(R, f)$

The 'path' model is set to account for response of the path between source and site (not including the local site response). 

#### $$P(R, f) = Z(R) N(R, f) \tag{6}$$

It can be broken down into its two constituent parts:
1. Geometric spreading - $Z(R)$
2. Aneslatic attenuation - $N(R, f)$

#### Geometric spreading

The theoretical model for geometric spreading of P or S body waves in a wholespace is $1/R$ ($R$ is distance in $\mathrm{km}$); which for simplicity we will assume in our computations. However, we note that in practice this is rarely observed and we typically resolve 'apparent' geometric spreading functions. These functions capture regionally observed variations in the observerd geometric spreading functions. It should be noted that such models are rarely derived independently from an anelastic model, and as seen in $\mathrm{Eq. 6}$, these components can trade-off with each other. You should never use an apparent geometric spreading model independently from is companion anelastic model if it has one. Note that geometrical spreading is **frequency independent**.

From Boore (2003) you can represent an apparent geometrical spreading model with the following general piecewise function:

$$ 
     Z(R)=\left\{
        \begin{array}{ll}
             \frac{R}{R_0} & R \leq R_1 \\
             Z(R_1) \left( \frac{R_1}{R} \right)^{p_1} & R_1 \leq R \leq R_2 \\
             \vdots \\
             Z(R_n) \left( \frac{R_n}{R} \right)^{p_n} & R_n \leq R \\
         \end{array} 
         \right. \tag{7}
$$
         
Here, R is distance ($\mathrm{km}$), $\mathrm{n}$ is the number of linear segments and $\mathrm{p}$ is the exponent of segment $\mathrm{n}$.       


In [4]:
@interact
def plotgeo(p=(0.5, 2, 0.1)):
    fig, ax = plt.subplots(1, 1, figsize=(8, 7))
    R = np.geomspace(1, 400, 40)
    for _p in np.arange(0.5, 2.5, 0.5):
        ax.plot(R, sm.models.single_geospreading(R, p=_p), label=f"p={_p:.1f}")
        ax.legend()
    Y = sm.models.single_geospreading(R, p)
    ax.semilogx(R, Y, 'k--', lw=2)
    ax.set_xlabel("R [km]")
    ax.set_ylabel(r"$\mathrm{\log_{10}}Z(R)$")

interactive(children=(FloatSlider(value=1.2000000000000002, description='p', max=2.0, min=0.5), Output()), _do…

#### Anelastic attenuation

We use a typical and simplified model of anelastic attenuation which accounts for both intrinsic and scattering attenuation of waves. This model is essentially a smooth high-pass filter which increases with distance and (or) frequency. 

$$N(R, f) = \mathrm{exp} \left[ \frac{-\pi f R}{Q_{(p / s)}(f) c_Q} \right] \tag{8}$$

Here, $c_Q$ is the averege wavespeed of the path for the phase of interest and $Q_{(p / s)}(f)$ is a frequency dependent quality factor for p or s waves along the path: 

$$Q(f) = Q_0f^{(1-a)},\tag{9}$$

and $a$ is a sensitivity parameter which tunes the degree of attenutation over frequency. $Q$ can be assumed **independent of frequency** ($a=1$) to leave $Q_0$ - the frequency independent component of $Q(f)$ - which we will do here for simpicity. 

In [5]:
@interact
def plotanelastic(Q=[100, 10, 1000], b=[2400, 2600, 3200, 3600], R=(0, 425, 25)):
    
    f = np.geomspace(0.1, 50, 50)
    fig, ax = plt.subplots(1,1,figsize=(8, 7))
    
    for _R in [1, 100, 400]:
            ax.plot(f, sm.models.f_idep_attenutation(f, 100, _R, 3600), label=f"Q=100 | Vs=3600 | R={_R}")
            
    ax.semilogx(f, sm.models.f_idep_attenutation(f, Q, R, b), 'k--', lw=2)
    ax.set_xlabel("Freq. [Hz]")
    ax.set_ylabel(r"$\mathrm{\log_{10}} A(f, R)$")
    ax.legend()
    

interactive(children=(Dropdown(description='Q', options=(100, 10, 1000), value=100), Dropdown(description='b',…

### Site model: $G(f)$ - Under construction ...

The 'site' model is set to account for the local site attenuation and resonance.

### Simplified target spectrum 
Here we will combine source and path models and experiment with how the choice of earthquake scenario changes our target spectrum. For now we assume our site and instrument response are unity, $G(f)=1$ and $I(f)=1$. 

Eq.1 then becomes: 

$$Y(M_{0}, R, f) = E(M_{0}, f)P(R,f) \tag{10}$$



In [6]:
@interact
def plot_target(motion=['acc', 'vel', 'disp'], 
                sd=[1, 0.1, 100], 
                mw=(2, 7, 1), 
                vs=(2400, 4200, 400),
                rho=(2000, 3000, 200),
                p=(0.5, 2, 0.1),
                Q=[1000, 500, 250], 
                b=[2.4, 2.6, 3.2, 3.6], 
                R=(25, 425, 25)
                ):
    
    mm = {"acc": "[m/s/s] s", "vel": "[m/s] s", "disp": "m s"}
    
    fig, ax = plt.subplots(1,1, figsize=(8, 6))
    
    f = np.geomspace(0.01, 50, 50)
    fac = sm.models.motion_factor(f, motion=motion)
    mo = sm.eqphysics.mo_from_mw(mw)
    fc = sm.eqphysics.fc(vs, sd*(1E6), mo)
    E = sm.models.source_scf(f, np.log10(mo), fc, 1, 2)
    Z = sm.models.single_geospreading(R, p)
    A = sm.models.f_idep_attenutation(f, Q, R, b)
    MO_TO_DISP = sm.eqphysics.moment_scaling(vs, rho)
    
    Y = fac + E - MO_TO_DISP + Z + A
    
    ax.loglog(f, 10**Y, 'k--', lw=2, label=r"Y(M0, R, f)")
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel(r"Spectral Amp. {}".format(mm[motion]))
    ax.set_title("Target Spectrum")

interactive(children=(Dropdown(description='motion', options=('acc', 'vel', 'disp'), value='acc'), Dropdown(de…